# Тетрадка со сбором данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Getting the proza corpus from the archive

In [ ]:
corpus_path = '/content/drive/MyDrive/proza_ru.tar.gz'

In [ ]:
! tar -zxf /content/drive/MyDrive/proza_ru.tar.gz --directory /content/drive/MyDrive/kursovaya2022/corpora/proza_ru

In [ ]:
! ls /content/home/tsha/proza_ru/texts/2017/11

20171121034.txt  20171121732.txt


In [ ]:
! cp -R /content/home/tsha/proza_ru/texts/2017/11 /content/drive/MyDrive/kursovaya2022/

In [ ]:
! sudo du -sh /content/drive/MyDrive/kursovaya2022/

^C


In [ ]:
%cd /content/drive/MyDrive/kursovaya2022/

/content/drive/MyDrive/kursovaya2022


In [ ]:
%cd 11

/content/drive/MyDrive/kursovaya2022/11


# preprocessing

In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from abc import abstractmethod
import csv
import os
from bs4 import BeautifulSoup
import re
from nltk.tokenize import word_tokenize
import pymorphy2
from tqdm import tqdm
import nltk
nltk.download('punkt')

morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
dirs = os.listdir('/content/drive/MyDrive/kursovaya2022/corpora/proza_ru/home/tsha/proza_ru/texts/2017')

In [ ]:
files = []
for d in dirs:
  d = '/content/drive/MyDrive/kursovaya2022/corpora/proza_ru/home/tsha/proza_ru/texts/2017/' + d
  for f in os.listdir(d):
    files.append(d + '/' + f)

In [ ]:
len(files)

116269

preprocessing using code by hseling-api-direct-speech:

Defining the parser that extracts direct speech (code from https://github.com/hseling/hseling-api-direct-speech)

In [ ]:
# step.py

class PipelineStep:
    def __init__(self):
        pass

    @abstractmethod
    def annotate(self, text):
        pass

    def make_dict(self, list_of_inside_tag_strings, fucntion):
        dictionary = {string: fucntion(string)
                      for string in list_of_inside_tag_strings}
        return dictionary

    def read_csv(self, path, sep):
        # template = os.path.join(os.path.abspath(
        #     os.path.dirname(os.path.dirname(__file__))), path)
        return csv.reader(open(path, "r", encoding="utf-8"), delimiter=sep)

    def read_dict_csv(self, path, sep):
        # template = os.path.join(os.path.abspath(
        #     os.path.dirname(os.path.dirname(__file__))), path)
        reader = csv.DictReader(open(path, "r", encoding='utf-8-sig'),
                                delimiter=sep)
        return list(reader)

    def read_xml(self, text):
        return BeautifulSoup('<text>' + text + '</text>', "lxml")


In [ ]:
# quotes_processing.py

class QuotesAdapter(PipelineStep):
    QUOTES_OPEN = '«'
    QUOTES_CLOSE = '»'
    SPECIFIC_QUOTES_OPEN = '„“‘„'
    SPECIFIC_QUOTES_CLOSE = '“”’”'

    def __init__(self, path):
        super().__init__()
        self.__rule_df = self.read_csv(path, ';')

    def annotate(self, text):
        text = self.__replace_specific_quotes(text)
        return self.__process(text)

    def __replace_specific_quotes(self, text):
        text = text.replace(self.SPECIFIC_QUOTES_OPEN, self.QUOTES_OPEN)
        text = text.replace(self.SPECIFIC_QUOTES_CLOSE, self.QUOTES_CLOSE)
        return text

    def __process(self, text):
        for ind in self.__rule_df:
            rule = ind[0]
            quote = ind[1]
            text = self.__replace_quotes(rule, quote, text)
        return text

    def __replace_quotes(self, rule, result_quote, text):
        try:
            text = re.sub(rule, result_quote, text)
        except Exception as e:
            print(e)
            print(rule, "=" * 3, "ERROR IN REG EXP")
        return text


In [ ]:
# speech_detector.py

class SpeechDetector(PipelineStep):
    def __init__(self, path):
        super().__init__()
        self.rule_df = self.read_csv(path, ';')

    def annotate(self, text):
        for left, speech, right in self.rule_df:
            text = self.__find_speech(left, speech, right, text)
            text = re.sub('(<speech>)+', '<speech>', text)
            text = re.sub('(</speech>)+', '</speech>', text)
        return text

    def __compute_regex(self, left_context, speech, right_context):
        try:
            regex = re.compile("(" + left_context + ")" + "(" + speech + ")" +
                               "(" + right_context + ")",
                               flags=re.MULTILINE)
            return regex
        except Exception:
            print("EXCEPTION IN REG EXP:", left_context, speech, right_context)
            return "EXCEPTION IN REG EXP"

    def __find_speech(self, left_context, speech, right_context, text):
        rule = self.__compute_regex(left_context, speech, right_context)
        text = re.sub(rule, r"\g<1><speech>\g<2></speech>\g<3>", text)
        return text


In [ ]:
# file_reader.py

class FileReader(PipelineStep):

    def __init__(self):
        super().__init__()

    def annotate(self, text):
        return self.__delete_newlines(text)

    def __delete_newlines(self, text):
        text1 = re.sub(r' +', ' ', text)
        text2 = re.sub(r'\n(?P<cap> [а-яё])', r'\g<cap>', text1)
        text3 = re.sub(r'(?P<punc>[^!\?\.\:])\n', r'\g<punc>', text2)
        text4 = re.sub(r'[  ]+', ' ', text3)
        return text4


In [ ]:
# verb_tagger.py

class VerbTagger(PipelineStep):
    COMMENT = r'<author_comment>(.+?)</author_comment>'

    def __init__(self, path):
        super().__init__()
        self.__df_verbs = self.read_dict_csv(path, sep=';')

    def __make_new_comments(self, string):
        for word in word_tokenize(string):
            lemma = morph.parse(word)[0].normal_form
            regex = re.compile(r'((?<=[ \.:<>!-,])|^)' + '(' +
                               re.escape(word) + ')' + r'((?=[ \.:<>!-,]))')
            for line in self.__df_verbs:
                if lemma == line["verb"]:
                    string = re.sub(regex, '<speech_verb ' + 'semantic="' +
                                    str(line['semantic']) + '" emotion="' +
                                    str(line['emotion']) + '">' + word +
                                    '</speech_verb>', string)
        return string

    def __find_comments(self, text):
        comments = re.findall(self.COMMENT, text)
        return comments

    def annotate(self, text):
        comments = self.__find_comments(text)
        dictionary = self.make_dict(comments, self.__make_new_comments)
        for key in dictionary:
            try:
                text = re.sub(re.escape(key), dictionary[key], text)
            except:
              print('err')
        text = re.sub(r'(?P<verb><speech_verb.+?>)+', r'\g<verb>', text)
        text = re.sub('(</speech_verb>)+', '</speech_verb>', text)
        return text


In [ ]:
# pipeline.py

class Pipeline:
    def __init__(self, *args):
        self.__steps = list(args)

    def apply_to(self, text):
        for step in self.__steps:
            text = step.annotate(text)
        return text

    def add_step(self, step):
        self.__steps.append(step)


In [ ]:
# said_comment_tagger


class SaidCommentTagger(PipelineStep):
    SEPARATOR = r'»,?[ \u00A0][—–-]{1,2}[ \u00A0]|[.,!?…]' \
                r'[ \u00A0][—–-]{1,2}[ \u00A0]|:[ \u00A0]\n?«|' \
                r':[ \u00A0\n][—–-]{1,2}[ \u00A0]|»,[ \u00A0]'
    FIRST_IN_SAID = '-–—−«'
    SPEECH = r'<speech>(.+?)</speech>'
    SAID = "said"
    AUTHOR_COMMENT = "author_comment"

    def __init__(self):
        super().__init__()

    def __said_comment(self, string):
        lst = re.split("(" + self.SEPARATOR + ")", string)
        annotation_result_list = []
        if lst[0]:
            order = self.__define_order(lst[0][0])
            for index, st in enumerate(lst):
                tag = order[index % 3]['start']
                if tag == "<said>":
                    tag = tag.replace("<said>",
                                      "<said aloud='True' "
                                      "type='direct'>")
                st_with_tag = tag + str(st) + order[index % 3]['end']
                annotation_result_list.append(st_with_tag)
        else:
            pass
        return "".join(annotation_result_list)

    def __define_order(self, first_symbol):
        if first_symbol in self.FIRST_IN_SAID:
            return [{"start": '<{}>'.format(self.SAID),
                     "end": '</{}>'.format(self.SAID)},
                    {"start": '',
                     "end": ''},
                    {"start": '<{}>'.format(self.AUTHOR_COMMENT),
                     "end": '</{}>'.format(self.AUTHOR_COMMENT)}]
        else:
            return [{"start": '<{}>'.format(self.AUTHOR_COMMENT),
                     "end": '</{}>'.format(self.AUTHOR_COMMENT)},
                    {"start": '',
                     "end": ''},
                    {"start": '<{}>'.format(self.SAID),
                     "end": '</{}>'.format(self.SAID)}]

    def __get_speech(self, text):
        lst = re.findall(self.SPEECH, text)
        return lst

    def annotate(self, text):
        speech_list = self.__get_speech(text)
        dictionary = self.make_dict(speech_list, self.__said_comment)
        for key in dictionary:
            try:
                text = re.sub(re.escape(key), dictionary[key], text)
            except:
              print('err')
        return text


Check if direct speech was found in the file

In [ ]:
def check_if_speech(text):
  res = re.search('<speech>', text)
  if res:
    return text
  else:
    return None

In [ ]:
def process_data(text):
    file_id = None
    result = ""
    reader = FileReader()
    quotes_adapter = QuotesAdapter("/content/drive/MyDrive/kursovaya2022/hseling-api-direct-speech/quotes.csv")
    speech_detector = SpeechDetector("/content/drive/MyDrive/kursovaya2022/hseling-api-direct-speech/speech.csv")
    said_comment_tagger = SaidCommentTagger()
    verb_tagger = VerbTagger("/content/drive/MyDrive/kursovaya2022/hseling-api-direct-speech/verbs.csv")
    pipeline = Pipeline(reader, quotes_adapter, speech_detector,
                        said_comment_tagger, verb_tagger)
    result = pipeline.apply_to(text)
    result = check_if_speech(result)
    if result:
        speech = re.findall('<speech>(.+?)</speech>', result)
        for s in speech:
          said = re.findall('<said.+?>(.+?)</said>', s)
          auth = re.findall('<author_comment>(.+?)</author_comment>', s)
          vrb = re.findall('<speech_verb.+?>(.+?)</speech_verb>', s)
          if len(said) == len(auth) == len(vrb):
            return result


In [ ]:
def get_data(filepath):
  with open(filepath, 'r', encoding='utf-8') as file:
    try:
      text = file.read()
      return text
    except UnicodeDecodeError as err:
      print(err)
      return ''

In [ ]:
%cd /content/drive/MyDrive/kursovaya2022/hseling-api-direct-speech

/content/drive/MyDrive/kursovaya2022/hseling-api-direct-speech


In [ ]:
# corpus = []
with open('/content/drive/MyDrive/kursovaya2022/hseling-api-direct-speech/2017_results.txt', 'a', encoding='utf-8') as file:
  for f in tqdm(range(39253, len(files))):
    text = get_data(files[f])
    # corpus.append(process_data(text))
    out = process_data(text)
    if out:
      file.write(out)
      file.write('\n=====\n')


 26%|██▋       | 20288/77016 [1:58:25<5:14:10,  3.01it/s]

err


 42%|████▏     | 32118/77016 [3:04:23<4:17:45,  2.90it/s]


KeyboardInterrupt: ignored

In [ ]:
with open('/content/drive/MyDrive/kursovaya2022/hseling-api-direct-speech/2017_results.txt', 'r', encoding='utf-8') as file:
  text = file.read()
  texts = text.split('\n=====\n')

In [ ]:
len(texts)

33249

In [ ]:
texts[-2]

'Если я ничего не перепутал в их норвежской географии FINAL VERSION \nВоркутинский Всегда езжу из Ярославля в Москву и обратно на Воркутинском поезде. Он удачно вписывается в мои будни и по времени и по удобствам. Прихожу на Ярославль Гл. (так в билете написано, - Гл.), а тут и Воркутинский приходит. Ехать приятно, не без некоторого провинциального шика. Удобно и думать всю дорогу. В эту поездку сначала анализировал, что же означает Гл. после слова Ярославль. Постепенно моя мысль перескочила на другие слова, начинающиеся с Гл. Подумал я и о глине.\n Пока я сидел и размышлял о глине, в мой вагон Воркутинского вошли с двух сторон где-то полсотни людей в красивой униформе. Главным начальником у них оказалась женщина приятной наружности и хороших манер. Она звонко и чётко рявкнула: «Всем оставаться на своих местах! Мы вас не тронем! Нам нужен только Ринат Барабуллин. Есть здесь такой» С одного из передних сидений бочком-бочком выскользнула тень и зашептала в ухо начальницы. Так как в вагон

Remove empty instances

In [ ]:
corpus = [x for x in corpus if x]

In [ ]:
len(corpus)

6931

In [ ]:
corpus[100]

' \n У этой палаты не было никакого номера. Это вообще и не палата вовсе. Это большая комната, заставленная узкими односпальными кроватями, спаренными по две. Между ними промежуток, где стоит табуретка. На неё кладётся верхняя одежда, если у кого таковая есть. На этих кроватях поочерёдно отдыхают, дремлют больные. Психи!\n Да! Это психушка, дурдом! Отделение дневного пребывания! Ох-хо-хо! Сколько нас тут -- психов!\n Когда утром открываются двери этого замечательного во всех отношениях заведения, медсестра приглашает пациентов:\n - Проходите в палату, пожалуйста. Занимайте кровати и идите получать лекарства. Не толпитесь: кроватей на всех хватит.\n Как бы не так! Кроватей не хватает. «Пациенты» стараются войти первыми, быстро подбегают к кровати и кладут на подушку кто полотенце, кто простынку. Всё застолблена кровать! Люди, которым не повезло с кроватями, сидят на стульях. Хоть тут повезло: удалось стул занять! А тем, кому не повезло ни с кроватью, ни со стулом, сидят на табуретках. Э

Save the preprocessed corpus

In [ ]:
with open('/content/drive/MyDrive/kursovaya2022/hseling-api-direct-speech/2017_results.txt', 'w', encoding='utf-8') as file:
  for c in corpus:
    file.write(c)
    file.write('\n=====\n')